In [6]:
# 🔧 1. 필수 패키지 설치
!pip install flask pyngrok pvlib requests -q

# 🔧 2. ngrok 인증 토큰 설정 (https://dashboard.ngrok.com/get-started)
from pyngrok import ngrok
ngrok.set_auth_token("2wkLwNFmi0HsIDUsD6Zrud1XX9j_uBvFbpYrP9EhrwHJKgdA")  # ★ 필수!

# 🌞 3. Flask 앱 + Leaflet 지도 실행
from flask import Flask, request, jsonify, render_template_string
import requests

app = Flask(__name__)

@app.route('/')
def index():
    return render_template_string("""
    <!DOCTYPE html>
    <html>
    <head>
      <meta charset="utf-8" />
      <title>태양광 발전량 지도 MVP</title>
      <meta name="viewport" content="width=device-width, initial-scale=1.0">
      <link rel="stylesheet" href="https://unpkg.com/leaflet/dist/leaflet.css" />
      <style>#map { height: 100vh; }</style>
    </head>
    <body>
    <div id="map"></div>

    <script src="https://unpkg.com/leaflet/dist/leaflet.js"></script>
    <script>
      const map = L.map('map').setView([36.5, 127.8], 7);
      L.tileLayer('https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', {
        attribution: '© OpenStreetMap contributors'
      }).addTo(map);

      function onMapClick(e) {
        const lat = e.latlng.lat.toFixed(5);
        const lon = e.latlng.lng.toFixed(5);
        fetch(`/get_pv_data?lat=${lat}&lon=${lon}`)
          .then(res => res.json())
          .then(data => {
            if (data.error) {
              L.popup().setLatLng(e.latlng).setContent("❌ 데이터 조회 오류").openOn(map);
            } else {
              L.popup().setLatLng(e.latlng)
                .setContent(
                  `📍 위도: ${lat}, 경도: ${lon}<br>` +
                  `☀️ 연평균 일사량: ${data.ghi} kWh/m²<br>` +
                  `⚡ 예상 발전량: ${data.energy} kWh/kWp`
                ).openOn(map);
            }
          });
      }

      map.on('click', onMapClick);
    </script>
    </body>
    </html>
    """)

@app.route('/get_pv_data')
def get_pv_data():
    lat = request.args.get('lat', type=float)
    lon = request.args.get('lon', type=float)

    url = (
        f'https://power.larc.nasa.gov/api/temporal/climatology/point'
        f'?parameters=ALLSKY_SFC_SW_DWN&community=RE&latitude={lat}&longitude={lon}&format=JSON'
    )

    res = requests.get(url).json()
    try:
        ghi = res['properties']['parameter']['ALLSKY_SFC_SW_DWN']['ANN']
    except KeyError:
        return jsonify({'error': 'GHI data not found'}), 500

    energy = ghi * 0.85  # 시스템 손실 고려 (15%)
    return jsonify({'ghi': round(ghi, 1), 'energy': round(energy, 1)})

# 🚀 4. ngrok 터널 열고 앱 실행
public_url = ngrok.connect(5000)
print(f"\n🌍 여기에 접속하세요: {public_url}\n")
app.run(port=5000)



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
t=2025-05-07T13:39:26+0900 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="failed to dial ngrok server with address \"connect.us.ngrok-agent.com:443\": dial tcp 3.16.250.205:443: i/o timeout"
t=2025-05-07T13:39:37+0900 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="failed to dial ngrok server with address \"connect.us.ngrok-agent.com:443\": dial tcp 3.16.250.205:443: i/o timeout"


PyngrokNgrokError: The ngrok process errored on start: failed to dial ngrok server with address "connect.us.ngrok-agent.com:443": dial tcp 3.16.250.205:443: i/o timeout.

In [4]:
!pip install flask pyngrok requests -q

from pyngrok import ngrok
ngrok.set_auth_token("2wkLwNFmi0HsIDUsD6Zrud1XX9j_uBvFbpYrP9EhrwHJKgdA")  # https://dashboard.ngrok.com/get-started/your-authtoken

In [5]:
import requests, time, json

def frange(start, stop, step):
    while start <= stop:
        yield round(start, 2)
        start += step

lat_range = list(frange(33.0, 38.0, 0.5))
lon_range = list(frange(126.0, 130.0, 0.5))

heat_data = []

for lat in lat_range:
    for lon in lon_range:
        url = (
            f'https://power.larc.nasa.gov/api/temporal/climatology/point'
            f'?parameters=ALLSKY_SFC_SW_DWN&community=RE&latitude={lat}&longitude={lon}&format=JSON'
        )
        try:
            res = requests.get(url).json()
            ghi = res['properties']['parameter']['ALLSKY_SFC_SW_DWN']['ANN']
            intensity = round(ghi / 1600, 3)  # 0~1 범위 정규화
            heat_data.append([lat, lon, intensity])
            print(f"{lat}, {lon} → GHI: {ghi}")
        except:
            print(f"❌ 실패: {lat}, {lon}")
        time.sleep(1)

# JSON 저장
with open("heat_data.json", "w") as f:
    json.dump(heat_data, f)

33.0, 126.0 → GHI: 3.78
33.0, 126.5 → GHI: 3.78
33.0, 127.0 → GHI: 3.91
33.0, 127.5 → GHI: 3.91
33.0, 128.0 → GHI: 3.93
33.0, 128.5 → GHI: 3.93
33.0, 129.0 → GHI: 3.79
33.0, 129.5 → GHI: 3.79
33.0, 130.0 → GHI: 3.79
33.5, 126.0 → GHI: 3.78
33.5, 126.5 → GHI: 3.78
33.5, 127.0 → GHI: 3.91
33.5, 127.5 → GHI: 3.91
33.5, 128.0 → GHI: 3.93
33.5, 128.5 → GHI: 3.93
33.5, 129.0 → GHI: 3.79
33.5, 129.5 → GHI: 3.79
33.5, 130.0 → GHI: 3.79
34.0, 126.0 → GHI: 3.89
34.0, 126.5 → GHI: 3.89
34.0, 127.0 → GHI: 3.98
34.0, 127.5 → GHI: 3.98
34.0, 128.0 → GHI: 4.04
34.0, 128.5 → GHI: 4.04
34.0, 129.0 → GHI: 3.88
34.0, 129.5 → GHI: 3.88
34.0, 130.0 → GHI: 3.84
34.5, 126.0 → GHI: 3.89
34.5, 126.5 → GHI: 3.89
34.5, 127.0 → GHI: 3.98
34.5, 127.5 → GHI: 3.98
34.5, 128.0 → GHI: 4.04
34.5, 128.5 → GHI: 4.04
34.5, 129.0 → GHI: 3.88
34.5, 129.5 → GHI: 3.88
34.5, 130.0 → GHI: 3.84
35.0, 126.0 → GHI: 3.83
35.0, 126.5 → GHI: 3.83
35.0, 127.0 → GHI: 3.9
35.0, 127.5 → GHI: 3.9
35.0, 128.0 → GHI: 4.0
35.0, 128.5 → GHI: 

In [6]:
from flask import Flask, send_file, render_template_string
from pyngrok import ngrok

app = Flask(__name__)

@app.route('/')
def index():
    return render_template_string("""
    <!DOCTYPE html>
    <html>
    <head>
      <meta charset="utf-8" />
      <title>태양광 히트맵</title>
      <meta name="viewport" content="width=device-width, initial-scale=1.0">
      <link rel="stylesheet" href="https://unpkg.com/leaflet/dist/leaflet.css" />
      <style>#map { height: 100vh; }</style>
    </head>
    <body>
    <div id="map"></div>

    <script src="https://unpkg.com/leaflet/dist/leaflet.js"></script>
    <script src="https://unpkg.com/leaflet.heat/dist/leaflet-heat.js"></script>

    <script>
      const map = L.map('map').setView([36.5, 127.8], 7);
      L.tileLayer('https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', {
        attribution: '© OpenStreetMap contributors'
      }).addTo(map);

      fetch('/heat_data.json')
        .then(res => res.json())
        .then(data => {
          L.heatLayer(data, {radius: 30}).addTo(map);
        });
    </script>
    </body>
    </html>
    """)

@app.route('/heat_data.json')
def heatmap_json():
    return send_file('heat_data.json', mimetype='application/json')

public_url = ngrok.connect(5000)
print(f"🔗 접속 링크: {public_url}")
app.run(port=5000)

🔗 접속 링크: NgrokTunnel: "https://84ac-34-74-176-200.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [07/May/2025 01:42:33] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/May/2025 01:42:34] "GET /heat_data.json HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/May/2025 01:42:34] "GET /favicon.ico HTTP/1.1" 404 -
